In [246]:
import pandas as pd
import numpy as np

In [247]:
df_original = pd.read_csv("dados/acidentes_naofatais.csv", encoding = "latin", sep = ";")

In [248]:
acidentes = (df_original
.rename({
    "Data do Acidente": "data",
    "Logradouro": "rua",
    "Município": "nm_municipio",
    "Veículos Envolvidos - Motocicleta": "motocicletas",
    "Pessoas Envolvidas - Leve": "leve",
    "Pessoas Envolvidas - Grave": "grave",
}, axis = 1)
.filter(["data", "nm_municipio", "rua", "grave", "leve", "motocicletas"])
.query(f"(grave > 0 or leve > 0) and motocicletas > 0 and nm_municipio == 'SAO PAULO'")
.assign(
    mes = lambda _: _.data.str[5:7], 
    ano = lambda _: _.data.str[0:4])
.replace({"rua": {"'": " "}}, regex = True)
)

acidentes.head()

,data,nm_municipio,rua,grave,leve,motocicletas,mes,ano
44,2023-09-30,SAO PAULO,RUA DAS FLECHAS,0,1,1,09,2023
50,2023-09-30,SAO PAULO,AVENIDA SANTOS DUMONT,0,1,1,09,2023
51,2023-09-30,SAO PAULO,AVENIDA RAIMUNDO PEREIRA DE MAGALHAES,0,1,1,09,2023
56,2023-09-30,SAO PAULO,AVENIDA JACUPESSEGO,0,1,1,09,2023
69,2023-09-30,SAO PAULO,SP 015,0,2,1,09,2023


In [249]:
avenidas = (acidentes
.groupby(["ano", "rua"])
.aggregate({"motocicletas": "count"})
.reset_index()
.sort_values(by = ["ano", "motocicletas"])
# .query("rua.str.contains('BANDEIRANTES')")
.query("ano == '2021' and ~ rua.str.contains('SP ') and ~ rua.str.contains('SPA ') and ~ rua.str.contains('BR ')")
.tail(80)
)
avenidas.tail()

,ano,rua,motocicletas
8951,2021,AVENIDA WASHINGTON LUIS,147
8901,2021,AVENIDA SAPOPEMBA,157
8157,2021,AVENIDA ARICANDUVA,159
9011,2021,ESTRADA DE ITAPECERICA,174
8910,2021,AVENIDA SENADOR TEOTONIO VILELA,175


In [250]:
df = (acidentes
 .query(f"rua in {avenidas.rua.to_list()}")
 .groupby(["ano", "mes", "rua"])
 .aggregate({"motocicletas": "count"})
 .rename({"motocicletas": "acidentes"}, axis = 1)
 .reset_index()
)

In [251]:
df

,ano,mes,rua,acidentes
0,2019,01,AVENIDA ALCANTARA MACHADO,2
1,2019,01,AVENIDA ANTONIO ESTEVAO DE CARVALHO,2
2,2019,01,AVENIDA ARICANDUVA,8
3,2019,01,AVENIDA ATLANTICA,2
4,2019,01,AVENIDA BRIGADEIRO FARIA LIMA,1
...,...,...,...,...
4247,2023,09,RUA DOUTOR LUIZ AYRES,2
4248,2023,09,RUA MARIA AMALIA LOPES AZEVEDO,1
4249,2023,09,RUA MELO FREIRE,3
4250,2023,09,RUA SADAMU INOUE,2


In [252]:
lentidao = pd.read_csv("dados/lentidaotrechos2022.csv", encoding="latin", sep = ";")

In [253]:
temp = lentidao.replace({"corredor": {"Av": "Avenida", ",": "", "/": " ", "\(": " ", "\)": " ", "\.": " "}}, regex = True)

In [254]:
banlist = ["AVENIDA", "ESTRADA", "DOS", "DE", "DO"]
dicio = {avenida: "|".join([palavra for palavra in avenida.split(" ") if palavra not in banlist and len(palavra) > 1]) for avenida in avenidas.rua.to_list()}

In [255]:
dicio_conversao = {}

In [256]:
_ = "AVENIDA CORONEL SEZEFREDO FAGUNDES"
# dicio_conversao.update({_: })
print(temp.query(f"corredor.str.upper().str.contains('{(dicio[_])}')").corredor.unique())

[]


In [257]:
dicio_conversao

{}

In [258]:
dicio_conversao

{}

In [243]:
import json

In [245]:
json.load(open('cruzamento_infosiga-lentidao.json'))

{'AVENIDA VEREADOR JOAO DE LUCA': 'João De Luca Ver'}

In [244]:
with open("cruzamento_infosiga-lentidao.json", "w") as outfile:
    outfile.write(json.dumps(dicio_conversao))

In [259]:
dicio_conversao = {}
for avenida, regex in dicio.items():
    valores = temp.query(f"corredor.str.upper().str.contains('{(regex)}')").corredor.unique()

    print(avenida, "->", regex)
    print(dict({0: "Sair", 1: "Nenhuma das opções"}, **{str(numero + 2): valores[numero] for numero in range(len(valores))}))



AVENIDA VEREADOR JOAO DE LUCA -> VEREADOR|JOAO|LUCA
{0: 'Sair', 1: 'Nenhuma das opções', '2': 'João De Luca Ver'}
RUA DA MOOCA -> RUA|DA|MOOCA
{0: 'Sair', 1: 'Nenhuma das opções', '2': 'Edgar Facó Avenida ', '3': 'Interlagos Avenida I', '4': 'Abraão de Morais Avenida Prof Imig', '5': 'Juntas Provisórias R das', '6': 'Salim Farah Maluf Avenida Tatuapé Pte', '7': 'Francisco Morato Avenida Prof', '8': 'Rebouças  Eusébio Matoso Avenida', '9': 'João Dias Avenida', '10': 'Bandeirantes Avenida dos', '11': 'Estado Avenida do - DEC CT', '12': 'Washington Luis Avenida', '13': 'Consolação R da', '14': 'Francisco Matarazzo Avenida', '15': 'Cruzeiro do Sul Pt e Avenida', '16': 'Max Feffer Túnel  Cidade Jardim ', '17': 'Cidade Jardim   Europa   Colômbia', '18': 'Nove de Julho Avenida', '19': 'Republica da Armenia Vd', '20': 'Vitor Manzini Avenida', '21': 'Guido Caloi Avenida', '22': 'Eusébio M Francisco Morato Avenida Prof', '23': 'Ibirapuera Avenida', '24': 'Aricanduva Avenida Elev Pt', '25': 'Quei